In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
from io_images import get_images_infoframe
import paramiko
import os 
from process_ims import get_ims_pixel_size
import tifffile
import numpy as np
import subprocess


Get chunk images .tiff

In [23]:
base_path = "/mnt/gdrive/ThNe/development_retina/"
# input_folder = "microglia/P15/F/L/" 
input_folder = "trila_fuse_mg"
output_folder = "traice"
# filename = "microglia_Age_P15_Sex_F_Side_L_Animal_1.tif"
# filename = "fused_tp_0_ch_2.tif"
# filename = "microglia_Age_P15_Sex_F_Side_L_Animal_1.tif"
filename = "microglia_Age_P21_Sex_F_Side_L_Animal_1_chunk_1_trick.tif"

In [ ]:
full_path = os.path.join(base_path, input_folder, filename)
# Load the TIFF file
with tifffile.TiffFile(full_path) as tif:
    data = tif.asarray()[8:72] 


In [ ]:
# import matplotlib.pyplot as plt
# from skimage import exposure

# # Define Z slices and crop region
# z_slices = range(5, 45, 3)
# x_range = slice(2500, 4000)
# y_range = slice(2500, 4000)

# # Plot each selected slice
# fig, axes = plt.subplots(1, len(z_slices), figsize=(15, 5))
# for ax, z in zip(axes, z_slices):
#     img = data[z, x_range, y_range]
#     tile_clahe = exposure.equalize_adapthist(img, clip_limit=0.1)

#     ax.imshow(tile_clahe, cmap='gray')
#     ax.set_title(f"Z={z}")
#     ax.axis('off')

# plt.tight_layout()
# plt.show()


In [11]:
pix_size = {
    'x': 0.339024328759285,
    'y': 0.339024328759285,
    'z': 0.9
}

In [12]:
size = data.shape
# Create new metadata
metadata = {
    # Physical dimensions in microns
    "ExtendMinX": 0,
    "ExtendMaxX": size[2] * pix_size['x'],
    "ExtendMinY": 0,
    "ExtendMaxY": size[1] * pix_size['y'],
    "ExtendMinZ": 0,
    "ExtendMaxZ": size[0] * pix_size['z'],
    # Dimensions in pixels
    "SizeX": size[2],
    "SizeY": size[1],
    "SizeZ": size[0],
}


# Save to new file
new_filename = "microglia_Age_P21_Sex_F_Side_L_Animal_1_chunk_1.tif"
new_path = os.path.join(base_path, input_folder, new_filename)

tifffile.imwrite(new_path, data, metadata=metadata)

print(f"Saved new file to: {new_path}")


Saved new file to: /mnt/gdrive/ThNe/development_retina/trila_fuse_mg/microglia_Age_P21_Sex_F_Side_L_Animal_1_chunk_1.tif


In [ ]:
# # project_path = "results/retina_example"
# infoframe = get_images_infoframe(win_path, 
#                                  extension='.tif',
#                                  conditions = [],
# )

In [18]:
save_path = os.path.join(base_path, output_folder)

In [19]:
save_path

'/mnt/gdrive/ThNe/development_retina/traice'

In [24]:
function = f'CUDA_VISIBLE_DEVICES=0 trAIce img2swc'

file_path_args = f'-ip "{os.path.join(base_path, input_folder, filename)}"'

cube_args = f'-wss "(128, 128, 16)" -wsb "(128, 128, 16)" -tp "(0.9, 0.9, 0.8)"'

save_args = f'-spd "{save_path}" -mp "{save_path}" -spsl "{save_path}" -bsp "{save_path}" -nw 1'

command = f'{function} {file_path_args} {cube_args} {save_args}'

In [25]:
print(command)

CUDA_VISIBLE_DEVICES=0 trAIce img2swc -ip "/mnt/gdrive/ThNe/development_retina/trila_fuse_mg/microglia_Age_P21_Sex_F_Side_L_Animal_1_chunk_1_trick.tif" -wss "(128, 128, 16)" -wsb "(128, 128, 16)" -tp "(0.9, 0.9, 0.8)" -spd "/mnt/gdrive/ThNe/development_retina/traice" -mp "/mnt/gdrive/ThNe/development_retina/traice" -spsl "/mnt/gdrive/ThNe/development_retina/traice" -bsp "/mnt/gdrive/ThNe/development_retina/traice" -nw 1


In [26]:
subprocess.run(command, shell=True, check=True)

=======Make mat files out of img/nd2 file=======
img_path: /mnt/gdrive/ThNe/development_retina/trila_fuse_mg/microglia_Age_P21_Sex_F_Side_L_Animal_1_chunk_1_trick.tif
/mnt/gdrive/ThNe/development_retina/trila_fuse_mg/microglia_Age_P21_Sex_F_Side_L_Animal_1_chunk_1_trick.tif
(7583, 5745, 64)
{'ExtendMinX': 0, 'ExtendMinY': 0, 'ExtendMinZ': 0, 'ExtendMaxX': 2570.821484981658, 'ExtendMaxY': 1947.6947687220925, 'ExtendMaxZ': 57.6, 'SizeX': 7583, 'SizeY': 5745, 'SizeZ': 64}
=======Make Masks Soma=======
Target pixel/micrometer: x=0.9, y=0.9, z=0.8
metadata: 0 - 0 - 0 - 2570.821484981658 - 1947.6947687220925 - 57.6 - 7583 - 5745 - 64
px2mu origin: 2.9496408227092834 - 2.9496408227092834 - 1.1111111111111112
2.9496408227092834
False
new size: 2313 - 1752 - 46
- Scaling the image to target ...
- Making 3d images from somas and traces of the cells ...
 Image size: (2313, 1752, 46) - SizeX: 2313 - SizeY: 1752 - SizeZ: 46
- Slicing the image and masks into windows ...


100%|██████████| 19/19 [00:00<00:00, 27207.84it/s]


=======Soma Detection Stage=======
Soma location: /mnt/gdrive/ThNe/development_retina/traice/microglia_Age_P21_Sex_F_Side_L_Animal_1_chunk_1_trick/soma/microglia_Age_P21_Sex_F_Side_L_Animal_1_chunk_1_trick
Loading dataset: /mnt/gdrive/ThNe/development_retina/traice/microglia_Age_P21_Sex_F_Side_L_Animal_1_chunk_1_trick/img ==> Loading
Dataset: /mnt/gdrive/ThNe/development_retina/traice/microglia_Age_P21_Sex_F_Side_L_Animal_1_chunk_1_trick ==> Loaded
Device: cuda
num_patches: 512
patch size: (16, 16, 2)


  0%|          | 0/798 [00:00<?, ?it/s]

Loaded model from checkpoint. Ignored keys: []
Loaded model from checkpoint. Ignored keys: []
Loaded model from checkpoint. Ignored keys: []


...

Skeletonizing Labels: 100%|██████████| 1/1 [00:00<00:00, 22.86it/s]


file_path /mnt/gdrive/ThNe/development_retina/traice/microglia_Age_P21_Sex_F_Side_L_Animal_1_chunk_1_trick/branch/116/branch.tif


Skeletonizing Labels: 100%|██████████| 1/1 [00:00<00:00, 31.45it/s]


SWC saved in /mnt/gdrive/ThNe/development_retina/traice/microglia_Age_P21_Sex_F_Side_L_Animal_1_chunk_1_trick/branch
====> Total Run Time: 0h 8m 15s


CompletedProcess(args='CUDA_VISIBLE_DEVICES=0 trAIce img2swc -ip "/mnt/gdrive/ThNe/development_retina/trila_fuse_mg/microglia_Age_P21_Sex_F_Side_L_Animal_1_chunk_1_trick.tif" -wss "(128, 128, 16)" -wsb "(128, 128, 16)" -tp "(0.9, 0.9, 0.8)" -spd "/mnt/gdrive/ThNe/development_retina/traice" -mp "/mnt/gdrive/ThNe/development_retina/traice" -spsl "/mnt/gdrive/ThNe/development_retina/traice" -bsp "/mnt/gdrive/ThNe/development_retina/traice" -nw 1', returncode=0)